In [3]:
import pandas as pd
import numpy as np
import math
import timeit
import statistics

start = timeit.default_timer()

#Read in level 1 solar plant locations [lon, lat, kWh/kWp yearly, average W/m2 yearly]
df_ren = pd.read_csv('Data/level1renewable.csv')

year = 2019
type = 'alkaline'
interest = 0.08    #[]
full_load_hours = 6000    #[hours/yr]
opex_factor_solar = 0.015     #[]
capex_wind = 1260    #[Eur/kW]
opex_wind = 8     #[Eur/MWh]
wind_efficiency = 0.4    #[]
blade = 50         #[m]
turbine_size = 2       #[MW]
h2_demand = 50        #[kt/yr]
elec_opex = 0.02      #[]
comp_elec = 4       #[kWh/kg H2]
other_capex_elec = 41.6  #[Eur/kW]
water_cost = 0.07     #[Eur/kg H2]

#Determination of solar parameters
if year == 2019:
    capex_solar = 700    #[Eur/kWp]
    solar_efficiency = 0.64        #[]
elif year == 2050:
    capex_solar = 445  #[Eur/kWp]
    solar_efficiency = 0.74       #[]

#Determination of electrolyser parameters
if type == 'alkaline':
    capex_extra = 2.47  # [Eur/kg h2]
    if year == 2019:
        capex_h2 = 1232    #[Eur/kW]
        lifetime = 8.56    #[years]
        electrolyser_efficiency = 0.67    #[]
    elif year == 2050:
        capex_h2 = 616     #[Eur/kW]
        lifetime = 14.27   #[years]
        electrolyser_efficiency = 0.75     #[]
elif type == 'SOEC':
    capex_extra = 2.47  # [Eur/kg h2]
    if year == 2019:
        capex_h2 = 4928  # [Eur/kW]
        lifetime = 2.28  # [years]
        electrolyser_efficiency = 0.77    #[]
    elif year == 2050:
        capex_h2 = 880  # [Eur/kW]
        lifetime = 9.98  # [years]
        electrolyser_efficiency = 0.84     #[]
else:
    capex_extra = 0.91  # [Eur/kg h2]
    if year == 2019:
        capex_h2 = 1584  # [Eur/kW]
        lifetime = 6.85  # [years]
        electrolyser_efficiency = 0.58     #[]
    elif year == 2050:
        capex_h2 = 792  # [Eur/kW]
        lifetime = 14.27  # [years]
        electrolyser_efficiency = 0.70    #[]
    #7% cost reduction per doubling in installed capacity

#Annualisation of capex function
def annualise(capex,interest,lifetime):
    annualised_capex = capex * (interest/(1-((1+interest)**(-lifetime))))
    return annualised_capex

#Calculate required size of electricity generation  (MWh/yr)
h2_demand_hourly = h2_demand * 1000 / full_load_hours     #[ton/hr]
elec_demand = h2_demand_hourly * 39/electrolyser_efficiency        #[MW]
elec_demand_yearly = h2_demand * 39/electrolyser_efficiency       #[MWh/yr]

#Calculate the capex of the solar array required
df_ren['Solar Array Size'] = elec_demand_yearly * 1000 / df_ren['Solar Energy Potential']    #[kWp]
df_ren['Solar CapEx'] = df_ren['Solar Array Size'] * capex_solar    #[Eur]

#Calculate the capex of wind turbines required
capex_turbine = turbine_size * capex_wind * 1000
df_ren['Wind Turbine Power'] = df_ren['Wind Power Density'] * wind_efficiency * (blade**2) * math.pi / 1e3     #[kW]
df_ren['No. of Turbines']  = elec_demand / df_ren['Wind Turbine Power']                     #[]
df_ren['Wind CapEx'] = df_ren['No. of Turbines'] * capex_turbine            #[Eur]

#Get minimum cost location from solar and wind and calculate cost/yr and cost/kWh
df_ren['Yearly Cost Solar'] = annualise(df_ren['Solar CapEx'],interest,25) + opex_factor_solar * df_ren['Solar CapEx']          #[Eur/yr]
df_ren['Yearly Cost Wind'] = annualise(df_ren['Wind CapEx'],interest,20) + opex_wind * elec_demand_yearly                       #[Eur/yr]
df_ren['Elec Cost Solar'] = df_ren['Yearly Cost Solar'] / elec_demand_yearly                                          #[Eur/MWh]
df_ren['Elec Cost Wind'] = df_ren['Yearly Cost Wind'] / elec_demand_yearly                                            #[Eur/MWh]
df_ren['Cheaper source'] = ['Solar' if x < y else 'Wind' for x, y in zip(df_ren['Yearly Cost Solar'], df_ren['Yearly Cost Wind'])]

#Calculate the cost of the electrolyser
total_capex_h2 = (capex_h2 + other_capex_elec) * elec_demand * 1000    #[Eur]
ann_capex_h2 = annualise(total_capex_h2, interest, lifetime)            #[Eur/yr]
yearly_cost_h2 = ann_capex_h2 + elec_opex * total_capex_h2 + (capex_extra + water_cost) * h2_demand * 1000 * 1000   #[Eur/yr]
electrolysis_per_kg_cost = yearly_cost_h2 / (h2_demand * 1000 * 1000)     #[Eur/kg H2]

#Calculate total generation cost/yr
df_ren['Yearly gen. cost'] = [min(x,y) + yearly_cost_h2 for x, y in zip(df_ren['Yearly Cost Solar'], df_ren['Yearly Cost Wind'])]      #[Eur/yr]
df_ren['Gen. cost per kg H2'] = df_ren['Yearly gen. cost'] / (h2_demand * 1000 * 1000)     #[Eur/kg H2]

stop = timeit.default_timer()

print('Time: ', stop - start)

Time:  0.04904170000003205


In [4]:
print('Min cost of solar is ' + str(min(df_ren['Elec Cost Solar'])) + ' Eur/MWh')
print('Min cost of wind is ' + str(min(df_ren['Elec Cost Wind'])) + ' Eur/MWh')
print('Max cost of solar is ' + str(max(df_ren['Elec Cost Solar'])) + ' Eur/MWh')
print('Max cost of wind is ' + str(max(df_ren['Elec Cost Wind'])) + ' Eur/MWh')
print('Avg cost of solar is ' + str(statistics.mean(df_ren['Elec Cost Solar'])) + ' Eur/MWh')
print('Avg cost of wind is ' + str(statistics.mean(df_ren['Elec Cost Wind'])) + ' Eur/MWh')
print('Median cost of solar is ' + str(statistics.median(df_ren['Elec Cost Solar'])) + ' Eur/MWh')
print('Median cost of wind is ' + str(statistics.median(df_ren['Elec Cost Wind'])) + ' Eur/MWh')

Min cost of solar is 31.957231057668075 Eur/MWh
Min cost of wind is 11.33396298421145 Eur/MWh
Max cost of solar is 104.52610837622532 Eur/MWh
Max cost of wind is 8735.079701239172 Eur/MWh
Avg cost of solar is 50.559076214536994 Eur/MWh
Avg cost of wind is 177.20732306388493 Eur/MWh
Median cost of solar is 48.24653875787869 Eur/MWh
Median cost of wind is 98.70667401633767 Eur/MWh


In [6]:
df_ren.sample(10)

,Longitude,Latitude,Solar Energy Potential,Wind Power Density,Solar Array Size,Solar CapEx,Wind Turbine Power,No. of Turbines,Wind CapEx,Yearly Cost Solar,Yearly Cost Wind,Elec Cost Solar,Elec Cost Wind,Cheaper source,Yearly gen. cost,Gen. cost per kg H2
5813,140.0,38.0,1209.975021,139.21793,2405.378385,1.683765e+06,437.366026,1.109082,2.794886e+06,182989.510201,3.079489e+05,62.873319,105.808072,Solar,2.419660e+08,4.839320
5230,106.0,21.0,1069.450024,119.04486,2721.443447,1.905010e+06,373.990458,1.297024,3.268501e+06,207034.205790,3.561876e+05,71.134830,122.382404,Solar,2.419901e+08,4.839801
3750,44.0,49.0,1246.110013,197.36966,2335.626656,1.634939e+06,620.055074,0.782309,1.971419e+06,177683.137283,2.240769e+05,61.050104,76.990529,Solar,2.419607e+08,4.839214
5013,97.0,40.0,1693.234978,322.08350,1718.868202,1.203208e+06,1011.855157,0.479391,1.208066e+06,130763.148258,1.463278e+05,44.928877,50.276730,Solar,2.419138e+08,4.838276
1805,-5.0,8.0,1427.150031,39.89068,2039.342534,1.427540e+06,125.320267,3.870680,9.754113e+06,155143.279660,1.016762e+06,53.305640,349.348839,Solar,2.419382e+08,4.838763
5715,128.0,37.0,1361.450007,72.64221,2137.755883,1.496429e+06,228.212233,2.125542,5.356365e+06,162630.089459,5.688412e+05,55.878031,195.448004,Solar,2.419456e+08,4.838913
2678,22.0,43.0,1315.825007,81.17099,2211.880566,1.548316e+06,255.006186,1.902207,4.793562e+06,168269.135527,5.115185e+05,57.815549,175.752510,Solar,2.419513e+08,4.839026
76,-117.0,45.0,1535.554919,135.81567,1895.371976,1.326760e+06,426.677511,1.136865,2.864899e+06,144190.698523,3.150799e+05,49.542445,108.258220,Solar,2.419272e+08,4.838544
3520,39.0,5.0,1543.950007,34.17856,1885.066063,1.319546e+06,107.375113,4.517570,1.138428e+07,143406.674716,1.182797e+06,49.273063,406.397045,Solar,2.419264e+08,4.838529
5389,112.0,47.0,1793.610013,440.81480,1622.675911,1.135873e+06,1384.860537,0.350270,8.826795e+05,123445.305769,1.131864e+05,42.414541,38.889701,Wind,2.418962e+08,4.837924


597611940.2985075